In [ ]:
import csv
import re
from docx import Document
import pandas as pd

#faili asukoht ja väljund
input_file = "katse_stahl2.docx"
output_file = "stahl_output_katse2.xlsx"


#regulaarsavaldised
#^ regex algab rea algusest
#(.*?) sõna või fraas (tänapäevane vaste)
#\s{2,} kahe või enama tühiku järjestus, eraldaja tulpade vahel
#('.*?')? tekst, mis on jutumärkide vahel, ? tähendab, et see on valikuline, st võib olla, aga ei pea
#\s{2,}(\w+) sõnaliigi eraldamiseks
#\s{2,}(¨)? sümbol • 
#\s{2,}(.*?) vana kirjakeele vaste
#\s{2,}(\w.*?\w)? saksakeelne vaste
#\s{2,}(\d+)$ leheküljenumber

regex = re.compile(r"^(.*?)\s{2,}('.*?')?\s{2,}(\w+)\s{2,}(¨)?\s{2,}(.*?)\s{2,}(\w.*?\w)?\s{2,}(\d+)$")
regex_nool = re.compile(r"^(.*?)\s+→\s+(.*?)$")  #read, mis sisaldavad noolt (→)

regex_symbol = re.compile(r"^(.*?)\s{2,}(\w+)\s{2,}(¨)?\s{2,}(.*?)\s{2,}(.*?)\s{2,}(\d+)$")

regex_kohanimed = re.compile(r"^(.*?)\s{2,}(\w+)\s+(.*?)\s{2,}(¨)\s+(.*?)\s+(.*?)\s{2,}(\d+)$")

regex_verbid = re.compile(r"^(.*?)\s{2,}(\w+)\s{2,}(¨)\s{2,}(.*?)\s{2,}(.*?)\s{2,}(\d+)$")

#andmete list
andmed = []

#doxc faili sisu lugemine
doc = Document(input_file)
for para in doc.paragraphs:
    line = para.text.strip()

    #kas rida vastab struktueeritud mustrile
    match = regex.match(line)
    if match:
        tekst_bold = match.group(1)
        tekst_jutumärkides = match.group(2) if match.group(2) else ""
        sõnaliik = match.group(3)
        vana_kirjakeel = match.group(5) if match.group(4) else ""
        saksakeelne = match.group(6)
        leheküljenumber = match.group(7)

        andmed.append({
            "tänapäevane vaste": tekst_bold,
            "tähendus": tekst_jutumärkides,
            "sõnaliik": sõnaliik,
            "vanas kirjakeeles": vana_kirjakeel,
            "saksakeelne vaste": saksakeelne,
            "leheküljenumber": leheküljenumber
        })
    else:
        # kas reas on nool (→)
        arrow_match = regex_nool.match(line)
        if arrow_match:
            # kui on, siis lisatakse see tänapäeva vaste tulpa (enne ja pärast noolt)
            tekst_bold = f"{arrow_match.group(1)} → {arrow_match.group(2)}"  #teksti ühendamine enne ja pärast noolt

            andmed.append({
                "tänapäevane vaste": tekst_bold,  # terve rida läheb "tänapäevane vaste" tulpa
                "tähendus": "",       # tekstis pole jutumärkide
                "sõnaliik": "",          #sõnaliik puudub real
                "vanas kirjakeeles": "",
                "saksakeelne vaste": "",         #saksakeelne vaste real puudub
                "leheküljenumber": ""
            })
            
        else:
            # Kas rida vastab sümboliga • mustrile
            symbol_match = regex_symbol.match(line)
            if symbol_match:
                tekst_bold = symbol_match.group(1)
                sõnaliik = symbol_match.group(2)
                vana_kirjakeel = symbol_match.group(4)
                saksakeelne = symbol_match.group(5)
                leheküljenumber = symbol_match.group(6)

                andmed.append({
                    "tänapäevane vaste": tekst_bold,
                    "tähendus": "",  
                    "sõnaliik": sõnaliik,
                    "vanas kirjakeeles": vana_kirjakeel,
                    "saksakeelne vaste": saksakeelne,
                    "leheküljenumber": leheküljenumber
                })
                
            else:
                #sümbol • ja •
                symbol_var_match = regex_symbol.match(line)
                if symbol_var_match:
                    tekst_bold = symbol_var_match.group(1)
                    tekst_jutumärkides = symbol_var_match.group(2) if symbol_var_match.group(2) else ""
                    sõnaliik = symbol_var_match.group(3)
                    vana_kirjakeel = symbol_var_match.group(5)
                    saksakeelne = symbol_var_match.group(6)
                    leheküljenumber = symbol_var_match.group(7)

                    andmed.append({
                            "tänapäevane vaste": tekst_bold,
                            "tähendus": tekst_jutumärkides,
                            "sõnaliik": sõnaliik,
                            "vanas kirjakeeles": vana_kirjakeel,
                            "saksakeelne vaste": saksakeelne,
                            "leheküljenumber": leheküljenumber
                    })
                    
                else:
                    regex_kohanimed_match = regex_kohanimed.match(line)
                    if regex_kohanimed_match:
                        tekst_bold = regex_kohanimed_match.group(1)
                        sõnaliik = regex_kohanimed_match.group(2)
                        tekst_jutumärkides = regex_kohanimed_match.group(3)
                        vana_kirjakeel = regex_kohanimed_match.group(5)
                        saksakeelne = regex_kohanimed_match.group(6)
                        leheküljenumber = regex_kohanimed_match.group(7)
                            
                        andmed.append({
                            "tänapäevane vaste": tekst_bold,
                            "tähendus": tekst_jutumärkides,
                            "sõnaliik": sõnaliik,
                            "vanas kirjakeeles": vana_kirjakeel,
                            "saksakeelne vaste": saksakeelne,
                             "leheküljenumber": leheküljenumber
                        })
                        
                    else:
                        regex_verbid_match = regex_verbid.match(line)
                        if regex_verbid_match:
                            tekst_bold = regex_verbid_match.group(1)
                            sõnaliik = regex_verbid_match.group(2)
                            vana_kirjakeel = regex_verbid_match.group(4)
                            saksakeelne = regex_verbid_match.group(5)
                            leheküljenumber = regex_verbid_match.group(6)
                            
                            andmed.append({
                                "tänapäevane vaste": tekst_bold,
                                "tähendus": "",
                                "sõnaliik": sõnaliik,
                                "vanas kirjakeeles": vana_kirjakeel,
                                "saksakeelne vaste": saksakeelne,
                                "leheküljenumber": leheküljenumber
                            })
                            

#csv faili salvestamine
with open(output_file, mode="w", encoding="utf-8", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["tänapäevane vaste", "tähendus", "sõnaliik", "vanas kirjakeeles", "saksakeelne vaste", "leheküljenumber"])
    writer.writeheader()
    writer.writerows(andmed)

    
#exceli faili salvestamine
df = pd.DataFrame(andmed)  #andmed DataFrame'iks
df.to_excel(output_file, index=False, engine='openpyxl')  #exceli fail


print(f"Andmed salvestati faili: {output_file}")


In [ ]:
import pandas as pd

# Faili laadimine

input_file = "stahl_output_katse2.xlsx"
output_file = "stahl_kikas2_grammatika.xlsx"

#exceli andmed DataFrame'ina
df = pd.read_excel(input_file)

# 
#vanas kirjakeeles jagame kaheks kooloni alusel
df[['St märksõna', 'St gram-vormid']] = df['vanas kirjakeeles'].str.split(':', n=1, expand=True)

#tulemuse salvestame exceli faili
df.to_excel(output_file, index=False)

print(f"Andmed on salvestatud faili: {output_file}")